<img src='letscodebr_cover.jpeg' align='left' width=100%/>

# Ada Tech [DS-PY-004] Técnicas de Programação I (PY) Aulas 4 e 5 : Pandas - Resolução do Exercício 4.

# Prática adicional sobre joins com pandas

Vamos trabalhar com um conjunto de dados [Coupon Redemption_SMOTE_ Feature Selection](https://www.kaggle.com/vasudeva009/coupon-redemption-smote-feature-selection/data) de campanha de marketing. É representado por um conjunto de tabelas relacionais:

* train: relaciona o id do consumidor, cupons de desconto, campanha de marketing e contém a variável de destino 'redemption_status', que informa se o cupom foi resgatado ou não.

* campaign_data: data de início e término da campanha e tipo de campanha

* customer_demographics: faixa etária, estado civil, tamanho da família, número de filhos, nível de renda, proprietário / inquilino / hipoteca

* coupon_item_mapping: cada cupom oferece benefícios em diferentes itens

* item_data: marca, tipo de marca e categoria do item

* customer_transaction_data: dados sobre compras feitas pelos consumidores. Não sabemos em qual compra utilizaram o cupom de desconto, talvez a promoção funcione para mais de uma compra.

<img src='img/Schema.png' align='center' width=40%/>

In [2]:
!pip install scikit-learn


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
from sklearn.impute import SimpleImputer


In [4]:
import numpy as np

from scipy.stats import mode

In [5]:
train = pd.read_csv("../data/train.csv")
cust_dem = pd.read_csv("../data/customer_demographics.csv")
trans = pd.read_csv("../data/customer_transaction_data.csv")
coupon_item = pd.read_csv("../data/coupon_item_mapping.csv")
item_data = pd.read_csv("../data/item_data.csv")
campaign_data = pd.read_csv("../data/campaign_data.csv")


## Exercício 1 - Train dataframe

* Olhe para o dataframe e verifique se há valores nulos.

In [6]:
train.isna().sum()

id                   0
campaign_id          0
coupon_id            0
customer_id          0
redemption_status    0
dtype: int64

## Exercício 2 - Customer demographics dataframe

* Verifique se há nulos
* Olhe para os dtypes
* Impute valores nulos a partir do valor modal de cada coluna
* Traga as colunas de variáveis categóricas para o dtype 'category'

In [7]:
cust_dem.isna().sum()

customer_id         0
age_range           0
marital_status    329
rented              0
family_size         0
no_of_children    538
income_bracket      0
dtype: int64

In [8]:
cust_dem.no_of_children.value_counts()

no_of_children
1     107
3+     60
2      55
Name: count, dtype: int64

In [9]:
print(cust_dem.dtypes)

imputer = SimpleImputer(strategy="most_frequent")

cust_dem = pd.DataFrame(imputer.fit_transform(
    cust_dem), columns=cust_dem.columns)

cust_dem = cust_dem.assign(
    age_range=lambda d: d["age_range"].astype("category"),
    marital_status=lambda d: d["marital_status"].astype("category"),
    rented=lambda d: d["rented"].astype("category"),
    family_size=lambda d: d["family_size"].astype("category"),
    no_of_children=lambda d: d["no_of_children"].astype("category"),
    income_bracket=lambda d: d["income_bracket"].astype("category"),
)

cust_dem

customer_id        int64
age_range         object
marital_status    object
rented             int64
family_size       object
no_of_children    object
income_bracket     int64
dtype: object


,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket
0,1,70+,Married,0,2,1,4
1,6,46-55,Married,0,2,1,5
2,7,26-35,Married,0,3,1,3
3,8,26-35,Married,0,4,2,6
4,10,46-55,Single,0,1,1,5
...,...,...,...,...,...,...,...
755,1577,36-45,Married,0,2,1,5
756,1578,46-55,Married,0,3,1,6
757,1579,46-55,Married,0,1,1,4
758,1580,26-35,Married,0,2,1,5


## Exercício 3 - Transactions dataframe

* Verifique se há nulos
* Veja os dtypes
* Remova a coluna `dates`

In [10]:
print(trans.isna().sum())

print(trans.dtypes)

trans = trans.drop(columns=["date"])


date               0
customer_id        0
item_id            0
quantity           0
selling_price      0
other_discount     0
coupon_discount    0
dtype: int64
date                object
customer_id          int64
item_id              int64
quantity             int64
selling_price      float64
other_discount     float64
coupon_discount    float64
dtype: object


## Exercício 4 - Coupon-Item dataframe
* verifique se há dados nulos

In [11]:
print(coupon_item.isna().sum())

coupon_id    0
item_id      0
dtype: int64


## Exercício 5 - Item dataframe

* Verifique se há nulos
* Converta colunas categóricas para o dtype `category`

In [12]:
print(item_data.isna().sum())

print(item_data.dtypes)


item_id       0
brand         0
brand_type    0
category      0
dtype: int64
item_id        int64
brand          int64
brand_type    object
category      object
dtype: object


In [13]:
item_data = item_data.assign(
    brand_type=lambda d: d["brand_type"].astype("category"),
    category=lambda d: d["category"].astype("category"),
)


## Exercício 6 - Campaign dataframe
* Verifique se há valores nulos.
* Converta as colunas `start_date` e `end_date` para `TimeStamp`.
* Calcule o período das promoções como `start_date` - `end_date`.
* Remova `start_date` e `end_date`.

In [14]:
print(campaign_data.isna().sum())

campaign_id      0
campaign_type    0
start_date       0
end_date         0
dtype: int64


In [15]:
campaign_data = (
    campaign_data.assign(
        start_date=pd.to_datetime(campaign_data["start_date"]),
        end_date=pd.to_datetime(campaign_data["end_date"]),
    )
    .assign(duration=lambda d: (d["end_date"] - d["start_date"]).dt.days)
    .drop(columns=["start_date", "end_date"])
)


C:\Users\levyv\AppData\Local\Temp\ipykernel_6620\2934144244.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  start_date=pd.to_datetime(campaign_data["start_date"]),
C:\Users\levyv\AppData\Local\Temp\ipykernel_6620\2934144244.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  end_date=pd.to_datetime(campaign_data["end_date"]),


In [16]:
campaign_data.head(20)

,campaign_id,campaign_type,duration
0,24,Y,60
1,25,Y,32
2,20,Y,130
3,23,Y,97
4,21,Y,32
5,22,X,32
6,18,X,-181
7,19,Y,32
8,17,Y,32
9,16,Y,32


## Exercício 7 - Merge Dataframes

1. Faça um merge entre train e campaign_data. Qual coluna fica em `on`? Quais são os critérios em `how`?

2. Combine o resultado do exercício $1$) com customer_demographics. Qual coluna fica em 'on'? Quais são os critérios em 'how'?

3. Verifique os valores nulos obtidos em $2$.

4. Imputamos os valores ausentes nas colunas demográficas. Para isso, amostramos com reposição da distribuição conjunta dos atributos demográficos observados.

5. Merge com transações médias por assunto:

* Faça um groupby por consumidor no dataframe da transação

* Adicione o valor médio às variáveis contínuas (`selling_price`, `quantity`, etc)

* merge com o DataFrame obtido em 4. Qual coluna fica em `on`? Qual critério entra em `how`?

6. Merge com transações médias por item

* a. Faça um agrupamento por item no dataframe da transação

* b. Calcule o valor médio de variáveis contínuas

* c. Faça um merge com o dataframe `coupon_item`

* d. Faça um merge de c. com item_data. Qual coluna fica em `on`? Quais são os critérios em `how`?

* e. Faça um agrupamento do dataframe d.: agrupando por `coupon_id`. Para cada grupo, calcule a moda das variáveis categóricas 'brand','brand_type,'category'.

* f. Faça um agrupamento do dataframe d. agrupando por `coupon_id`. Para cada grupo, calcule a média das variáveis contínuas.

* g. Faça uma fusão de d. com e. e então com f. Qual coluna fica em `on`? Quais são os critérios em `how`?

7. Salve o dataframe resultante em um csv para prática de classe.

In [17]:
trans_media_assunto = trans.groupby("customer_id", as_index=False).agg(
    selling_price=("selling_price", "mean"),
    quantity=("quantity", "mean"),
)

In [18]:
trans_media_items = trans.groupby("item_id", as_index=False).mean()

(coupon_item.merge(trans_media_items, on="item_id", how="left"))

,coupon_id,item_id,customer_id,quantity,selling_price,other_discount,coupon_discount
0,105,37,1044.0,1.0,47.73,-11.04,0.0
1,107,75,637.0,1.0,38.83,0.00,0.0
2,494,76,NaN,NaN,NaN,NaN,NaN
3,522,77,NaN,NaN,NaN,NaN,NaN
4,518,77,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
92658,32,69268,NaN,NaN,NaN,NaN,NaN
92659,32,68502,NaN,NaN,NaN,NaN,NaN
92660,32,68612,NaN,NaN,NaN,NaN,NaN
92661,33,71390,NaN,NaN,NaN,NaN,NaN


In [19]:
(
    train.merge(campaign_data, on="campaign_id", how="left")
    .merge(cust_dem, on="customer_id", how="left")
    .merge(trans_media_assunto, on="customer_id", how="left")
)


,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,duration,age_range,marital_status,rented,family_size,no_of_children,income_bracket,selling_price,quantity
0,1,13,27,1053,0,X,-12,46-55,Married,0,1,1,5,119.771667,1.333333
1,2,13,116,48,0,X,-12,36-45,Married,0,2,1,3,639.854000,1.733333
2,6,9,635,205,0,Y,31,46-55,Married,0,2,1,7,105.111364,1.636364
3,7,13,644,1050,0,X,-12,NaN,NaN,NaN,NaN,NaN,NaN,47.595000,1.000000
4,9,8,1017,1489,0,X,77,46-55,Married,0,2,1,3,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78364,128587,8,71,1523,0,X,77,70+,Married,0,2,1,1,56.160000,1.000000
78365,128589,30,547,937,0,X,133,70+,Married,0,2,1,2,81.965556,1.277778
78366,128590,8,754,1004,0,X,77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78367,128592,13,134,71,0,X,-12,36-45,Married,0,5+,3+,4,NaN,NaN


6. Merge com transações médias por item

* a. Faça um agrupamento por item no dataframe da transação

* b. Calcule o valor médio de variáveis contínuas

* c. Faça um merge com o dataframe `coupon_item`

* d. Faça um merge de c. com item_data. Qual coluna fica em `on`? Quais são os critérios em `how`?

* e. Faça um agrupamento do dataframe d.: agrupando por `coupon_id`. Para cada grupo, calcule a moda das variáveis categóricas 'brand','brand_type,'category'.

* f. Faça um agrupamento do dataframe d. agrupando por `coupon_id`. Para cada grupo, calcule a média das variáveis contínuas.

* g. Faça uma fusão de d. com e. e então com f. Qual coluna fica em `on`? Quais são os critérios em `how`?

In [20]:
d = coupon_item.merge(trans_media_items, on="item_id", how="left").merge(
    item_data, on="item_id", how="left")
d.head()

,coupon_id,item_id,customer_id,quantity,selling_price,other_discount,coupon_discount,brand,brand_type,category
0,105,37,1044.0,1.0,47.73,-11.04,0.0,56,Local,Grocery
1,107,75,637.0,1.0,38.83,0.00,0.0,56,Local,Grocery
2,494,76,NaN,NaN,NaN,NaN,NaN,209,Established,Grocery
3,522,77,NaN,NaN,NaN,NaN,NaN,278,Established,Grocery
4,518,77,NaN,NaN,NaN,NaN,NaN,278,Established,Grocery


In [21]:
(
    d.groupby("coupon_id", as_index=False).agg(
        brand=("brand", lambda x: x.value_counts().idxmax()),
        brand_type=("brand_type", lambda x: x.value_counts().idxmax()),
        category=("category", lambda x: x.value_counts().idxmax()),
    )
)


,coupon_id,brand,brand_type,category
0,1,1475,Established,Natural Products
1,2,2084,Established,Grocery
2,3,278,Established,Grocery
3,4,544,Established,Grocery
4,5,5357,Established,Pharmaceutical
...,...,...,...,...
1111,1112,5177,Established,Grocery
1112,1113,5184,Established,Pharmaceutical
1113,1114,2416,Established,Pharmaceutical
1114,1115,2446,Established,Grocery
